In [1]:
import pandas as pd
import numpy as np
import polars as pl
import fastexcel

In [2]:
# Read in file
df = pl.read_excel("data/Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2025.xlsx", 
                   sheet_name="Table 03",
                #    read_options={
                #     "skip_rows": 6,
                #     }
                   )
# Column names change
# df.columns = ["GCCSA code",	"GCCSA name", "SA4 code", "SA4 name", "SA3 code",
#               "SA3 name", "SA2 code", "SA2 name","2023 Pop", "2024 Pop",
#               "2023-24 Change", "2023-24 Change %", "Natural Increase", "Net internal migration",
#               "Net oveseas migration","Area","Population Density"]

In [3]:
df.head()

Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
i64,str,str,i64,str,str,str,str,i64
2025,"""March""","""Alpine""",3691,"""Coral Bank""","""D Public order and security of…","""D10 Weapons and explosives off…","""D11 Firearms offences""",1
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offen…","""B10 Arson""","""B19 Other fire related offence…",1
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offen…","""B30 Burglary/Break and enter""","""B322 Non-residential non-aggra…",2
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offen…","""B40 Theft""","""B42 Steal from a motor vehicle""",1
2025,"""March""","""Alpine""",3691,"""Dederang""","""B Property and deception offen…","""B40 Theft""","""B49 Other theft""",1


In [39]:
pl.Config.set_fmt_str_lengths(50)
pl.Config.set_tbl_rows(30)
df_2024=(
    df
        .filter(pl.col("Year") == 2024)
        # .filter(pl.col("Offence Subdivision") == "B40 Theft")

        .group_by(["Year","Local Government Area","Postcode", "Suburb/Town Name", "Offence Division", 'Offence Subdivision'])
        .agg(
            pl.sum("Incidents Recorded")
        )
        .filter(pl.col("Local Government Area") == 'Darebin')
        .sort("Incidents Recorded", descending=True)
        .rename({"Incidents Recorded" : "Incidents Recorded 2024"})
        [:,1:7] # select relevant columns
)
df_2024.write_csv("data/df_crime_2024.csv")

In [40]:
df_2025=(
    df
        .filter(pl.col("Year") == 2025)
        # .filter(pl.col("Offence Subdivision") == "B40 Theft")

        .group_by(["Year","Local Government Area","Postcode", "Suburb/Town Name", "Offence Division", 'Offence Subdivision'])
        .agg(
            pl.sum("Incidents Recorded")
        )
        .filter(pl.col("Local Government Area") == 'Darebin')
        .sort("Incidents Recorded", descending=True)
        .rename({"Incidents Recorded" : "Incidents Recorded 2025"})
        [:,1:7] # select relevant columns
)
df_2025.write_csv("data/df_crime_2025.csv")

In [ ]:
(
    df_2024
    .join(
        df_2025,
        on = ['Postcode','Suburb/Town Name','Offence Subdivision'],
        how = 'left'
    )
    .select(["Local Government Area","Postcode", "Suburb/Town Name", 
             "Offence Division", 'Offence Subdivision',"Incidents Recorded 2024",
             "Incidents Recorded 2025"])
    .filter(
        pl.col("Local Government Area") == 'Darebin'
    )
    
)

Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Incidents Recorded 2024,Incidents Recorded 2025
str,i64,str,str,str,i64,i64
"""Darebin""",3072,"""Preston""","""B Property and deception offences""","""B40 Theft""",1767,3006
"""Darebin""",3073,"""Reservoir""","""B Property and deception offences""","""B40 Theft""",1423,2172
"""Darebin""",3070,"""Northcote""","""B Property and deception offences""","""B40 Theft""",633,988
"""Darebin""",3071,"""Thornbury""","""B Property and deception offences""","""B40 Theft""",509,832
"""Darebin""",3073,"""Reservoir""","""E Justice procedures offences""","""E20 Breaches of orders""",447,389
"""Darebin""",3072,"""Preston""","""E Justice procedures offences""","""E20 Breaches of orders""",401,249
"""Darebin""",3073,"""Reservoir""","""A Crimes against the person""","""A20 Assault and related offences""",341,411
"""Darebin""",3073,"""Reservoir""","""B Property and deception offences""","""B20 Property damage""",308,339
"""Darebin""",3072,"""Preston""","""B Property and deception offences""","""B20 Property damage""",306,405
